<img style="float: left;;" src='Figures/iteso.jpg' width="100" height="200"/></a>

# <center> <font color= #000047> Variables Categóricas Ordinales </font> </center>

In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv('temps.txt',sep='\s+',header=None) # Fuente: http://astro.iam.udg.mx/Estacion/NOAAPRMO.TXT
                                                         #         http://astro.iam.udg.mx/Estacion/NOAAMO.TXT
data.columns=['Temperatura máxima','Temperatura mínima','HDD','CDD','Rain']
d=['Sábado','Domingo','Lunes','Martes','Miércoles','Jueves','Viernes']
dias=[d[i%7] for i in range(len(data))]
catego=['Soleado','Mayormente soleado','Parcialmente nublado','Mayormente nublado','Nublado','Lluvioso']
prono=[catego[-1] if data['Rain'][i]>0 else catego[int(round(1.2*(data['HDD'][i]-data['CDD'][i])+2))] for i in range(len(data))]
df=pd.DataFrame(columns=['Días','Pronóstico','Temperatura máxima','Temperatura mínima'],index=range(len(data)))
df.iloc[:,2:4]=data.iloc[:,0:2]
df['Días']=dias
df['Pronóstico']=prono
df.head()

# Codificación lineal

In [ ]:
#@title Codificación para la variable `Días`
n=len(df['Días'].unique())
lim=n-1 if n%2==0 else (n-1)//2
step = 2 if n%2==0 else 1
x=range(-lim,lim+1,step)
list(x)

In [ ]:
catego=df['Días'].unique()
catego

In [ ]:
Días_ord=np.concatenate([catego[2:],catego[:2]])
Días_ord

In [ ]:
a0=0
a1=5 #-0.1
Días_lin={categorías:a1*i+a0 for categorías,i in zip(Días_ord,x)}
Días_lin

In [ ]:
df['Días_lin']=df['Días'].map(Días_lin)
df.head()

In [ ]:
#@title Codificando la variable `Pronóstico`
n=len(df['Pronóstico'].unique())
lim=n-1 if n%2==0 else (n-1)//2
step=2 if n%2==0 else 1
x=range(-lim,lim+1,step)
list(x)

In [ ]:
cat=df['Pronóstico'].unique()
cat

In [ ]:
Pronóstico_ord=np.concatenate([cat[-3:-5:-1],
                               np.array([cat[0],cat[-1],cat[1],cat[-2]])
                               ])
Pronóstico_ord

In [ ]:
a0=0
a1=-1
Pronóstico_lin={categorías:a1*i+a0 for categorías,i in zip(Pronóstico_ord,x)}
Pronóstico_lin

In [ ]:
df['Pronóstico_lin']=df['Pronóstico'].map(Pronóstico_lin)
df.head()

# Codificación cuadrática

In [ ]:
#@title Codificación de la variable `Pronóstico`
a2=-1
a1=0
a0=-a2/n*np.sum([i**2 for i in x])
Pronóstico_cuad={categorías:a0+a1*i+a2*i**2 for categorías,i in zip(Pronóstico_ord,x)}
Pronóstico_cuad

In [ ]:
np.sum(list(Pronóstico_lin.values()))

In [ ]:
df['Pronóstico_cuad']=df['Pronóstico'].map(Pronóstico_cuad)
df.head()

In [ ]:
#@title Codificación de `Días`
n=len(df['Días'].unique())
lim=n-1 if n%2==0 else (n-1)//2
step=2 if n%2==0 else 1
x=range(-lim,lim+1,step)
a2=1
a1=0
a0=-a2/n*np.sum([i**2 for i in x])
Días_cuad={categos:a0+a1*i+a2*i**2 for categos,i in zip(Días_ord,x)}
Días_cuad

In [ ]:
np.sum(list(Días_cuad.values()))

In [ ]:
df['Días_cuad']=df['Días'].map(Días_cuad)
df.head()

# Regresión lineal

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(df['Pronóstico_lin'],df['Temperatura mínima'])
plt.grid()

In [ ]:
plt.scatter(df['Pronóstico_cuad'],df['Temperatura mínima'])
plt.grid()

In [ ]:
lin=LinearRegression()
lin.fit(df[['Pronóstico_lin','Pronóstico_cuad']],df['Temperatura mínima'])

In [ ]:
f=lin.predict(df[['Pronóstico_lin','Pronóstico_cuad']])
plt.scatter(df['Pronóstico_lin'],df['Temperatura mínima'],label='Observaciones')
plt.scatter(df['Pronóstico_lin'],f,label='Predicción')
plt.legend()
plt.grid()

In [ ]:
plt.scatter(df['Pronóstico_cuad'],df['Temperatura mínima'],label='Observaciones')
plt.scatter(df['Pronóstico_cuad'],f,label='Predicción')
plt.legend()
plt.grid()